In [29]:
import logging
import datetime
import pandas as pd
import os
from entsoe import EntsoePandasClient
from entsoe.exceptions import NoMatchingDataError

In [30]:
# Set up logging
logging.basicConfig(filename='entsoe.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logging.info('Script started at %s', pd.Timestamp.now(tz='UTC'))

In [31]:
#Set API keys
API_KEY='6276342c-e10c-4d88-8688-cb0a1cf163ca'

In [32]:
# Initialize the ENTSO-E client
client = EntsoePandasClient(api_key=API_KEY)

# Country codes
country_codes = ["DE_50HZ", "IT_NORD_SI", "AL", "IT_PRGP", "DE_AMPRION", "IT_ROSN", "AT", "IT_SARD", "BY", "IT_SICI", "BE", "IT_SUD", "BA", "RU_KGD", "BG", "LV", "CZ_DE_SK", "LT", "HR", "LU", "CWE", "MT", "CY", "ME", "CZ", "GB", "DE_AT_LU", "NL", "DE_LU", "NO_1", "DK", "NO_2", "DK_1", "NO_3", "DK_2", "NO_4", "DK_CA", "NO_5", "EE", "NO", "FI", "PL_CZ", "MK", "PL", "FR", "PT", "DE", "MD", "GR", "RO", "HU", "RU", "IS", "SE_1", "IE_SEM", "SE_2", "IE", "SE_3", "IT", "SE_4", "IT_SACO_AC", "RS", "IT_SACO_DC", "SK", "IT_BRNN", "SI", "IT_CNOR", "GB_NIR", "IT_CSUD", "ES", "IT_FOGN", "SE", "IT_GR", "CH", "IT_MACRO_NORTH", "DE_TENNET", "IT_MACRO_SOUTH", "DE_TRANSNET", "IT_MALTA", "TR", "IT_NORD", "UA", "IT_NORD_AT", "UA_DOBTPP", "IT_NORD_CH", "UA_BEI", "IT_NORD_FR", "UA_IPS"]

In [54]:
# Define the start and end time
start = pd.Timestamp("2023-07-07", tz="UTC")  # 1st January 2022 00:00:00 UTC
end = pd.Timestamp.now(tz="UTC")  # Current UTC time

In [34]:
sorted_country_codes = sorted(country_codes)
sorted_country_codes

['AL',
 'AT',
 'BA',
 'BE',
 'BG',
 'BY',
 'CH',
 'CWE',
 'CY',
 'CZ',
 'CZ_DE_SK',
 'DE',
 'DE_50HZ',
 'DE_AMPRION',
 'DE_AT_LU',
 'DE_LU',
 'DE_TENNET',
 'DE_TRANSNET',
 'DK',
 'DK_1',
 'DK_2',
 'DK_CA',
 'EE',
 'ES',
 'FI',
 'FR',
 'GB',
 'GB_NIR',
 'GR',
 'HR',
 'HU',
 'IE',
 'IE_SEM',
 'IS',
 'IT',
 'IT_BRNN',
 'IT_CNOR',
 'IT_CSUD',
 'IT_FOGN',
 'IT_GR',
 'IT_MACRO_NORTH',
 'IT_MACRO_SOUTH',
 'IT_MALTA',
 'IT_NORD',
 'IT_NORD_AT',
 'IT_NORD_CH',
 'IT_NORD_FR',
 'IT_NORD_SI',
 'IT_PRGP',
 'IT_ROSN',
 'IT_SACO_AC',
 'IT_SACO_DC',
 'IT_SARD',
 'IT_SICI',
 'IT_SUD',
 'LT',
 'LU',
 'LV',
 'MD',
 'ME',
 'MK',
 'MT',
 'NL',
 'NO',
 'NO_1',
 'NO_2',
 'NO_3',
 'NO_4',
 'NO_5',
 'PL',
 'PL_CZ',
 'PT',
 'RO',
 'RS',
 'RU',
 'RU_KGD',
 'SE',
 'SE_1',
 'SE_2',
 'SE_3',
 'SE_4',
 'SI',
 'SK',
 'TR',
 'UA',
 'UA_BEI',
 'UA_DOBTPP',
 'UA_IPS']

In [35]:
# Download and save the data for each country - 1
for country_code in sorted_country_codes:
    try:
        # Download actual total load
        actual_total_load = client.query_load(country_code=country_code, start=start, end=end)
        print(f"Actual Total Load for {country_code}:")
        print(actual_total_load)  # Print the data

        actual_total_load.index = actual_total_load.index.tz_convert('UTC').strftime('%Y-%m-%d %H:%M')
        actual_total_load = actual_total_load.replace(['N/A', 'n/e'], None) 
        actual_total_load.to_csv(f'{country_code}_actual_total_load.csv')
    except Exception as e:
        logging.error(f'Data extraction (actual_total_load) failed for {country_code}: {str(e)}')


In [ ]:
# Download day-ahead total load forecast - 2
for country_code in sorted_country_codes[1:5]:
    try:
        day_ahead_total_load_forecast = client.query_load_forecast(country_code=country_code, start=start, end=end)
        print(f"Day-Ahead Total Load Forecast for {country_code}:")
        print(day_ahead_total_load_forecast)  # Print the data

        day_ahead_total_load_forecast.index = day_ahead_total_load_forecast.index.tz_convert('UTC').strftime('%Y-%m-%d %H:%M')
        day_ahead_total_load_forecast = day_ahead_total_load_forecast.replace(['N/A', 'n/e'], None)
        day_ahead_total_load_forecast.to_csv(f'{country_code}_day_ahead_total_load_forecast.csv')
    except Exception as e:
        logging.error(f'Data extraction (day_ahead_total_load_forecast) failed for {country_code}: {str(e)}')

In [ ]:
# Download day-ahead total load forecast - 3
for country_code in sorted_country_codes:
    try:
        # Download installed generation capacity
        installed_capacity = client.query_installed_generation_capacity(country_code=country_code, start=start, end=end)
        print(f"Installed Generation Capacity for {country_code}:")
        print(installed_capacity)  # Print the data
        installed_capacity = installed_capacity.replace(['N/A', 'n/e'], None) 
        installed_capacity.index = pd.to_datetime(installed_capacity.index).year
        installed_capacity.to_csv(f'{country_code}_installed_generation_capacity.csv', index_label='Year')
    except Exception as e:
        logging.error(f'Data extraction (installed capacity) failed for {country_code}: {str(e)}')

In [51]:
psr_types = [
    "B01",  # Biomass
    "B02",  # Fossil Brown coal/Lignite
    "B03",  # Fossil Coal-derived gas
    "B04",  # Fossil Gas
    "B05",  # Fossil Hard coal
    "B06",  # Fossil Oil
    "B07",  # Fossil Oil shale
    "B08",  # Fossil Peat
    "B09",  # Geothermal
    "B10",  # Hydro Pumped Storage
    "B11",  # Hydro Run-of-river and poundage
    "B12",  # Hydro Water Reservoir
    "B13",  # Marine
    "B14",  # Nuclear
    "B15",  # Other
    "B16",  # Other renewable
    "B17",  # Solar
    "B18",  # Waste
    "B19",  # Wind Offshore
    "B20",  # Wind Onshore
]

In [52]:
# Create an empty DataFrame to store the results
all_data = pd.DataFrame()

In [50]:
# Loop through each country and psr_type
for country_code in sorted_country_codes:
    for psr_type in psr_types:
        try:
            # Download actual generation per production type
            actual_generation = client.query_generation(country_code=country_code, start=start, end=end, psr_type=psr_type)
            
            # Append the data to the all_data DataFrame
            all_data = all_data.concat(actual_generation)
            
            # Print the data for the current country and psr_type
            print(f"Actual Generation for {country_code} - {psr_type}:")
            print(actual_generation)
            
            # Save data to CSV file (optional)
            actual_generation.to_csv(f'{country_code}_{psr_type}_actual_generation.csv')
            
        except NoMatchingDataError:
            logging.warning(f'No data available for {country_code} - {psr_type} in the specified time range')
            continue  # Move to the next iteration
        
        except Exception as e:
            logging.error(f'Data extraction failed for {country_code} - {psr_type}: {str(e)}')

C:\Users\emmaa\AppData\Local\Temp\ipykernel_26936\1655986711.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(actual_generation)


Actual Generation for AT - B01:
                                    Biomass                   
                          Actual Aggregated Actual Consumption
2022-01-01 01:00:00+01:00             332.0                0.0
2022-01-01 01:15:00+01:00             332.0                0.0
2022-01-01 01:30:00+01:00             332.0                0.0
2022-01-01 01:45:00+01:00             332.0                0.0
2022-01-01 02:00:00+01:00             332.0                0.0
...                                     ...                ...
2023-07-14 13:15:00+02:00              76.0                0.0
2023-07-14 13:30:00+02:00              76.0                0.0
2023-07-14 13:45:00+02:00              76.0                0.0
2023-07-14 14:00:00+02:00              76.0                0.0
2023-07-14 14:15:00+02:00              80.0                0.0

[53714 rows x 2 columns]


C:\Users\emmaa\AppData\Local\Temp\ipykernel_26936\1655986711.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(actual_generation)


Actual Generation for AT - B04:
                                 Fossil Gas                   
                          Actual Aggregated Actual Consumption
2022-01-01 01:00:00+01:00              92.0                0.0
2022-01-01 01:15:00+01:00              90.0                0.0
2022-01-01 01:30:00+01:00              90.0                0.0
2022-01-01 01:45:00+01:00              92.0                0.0
2022-01-01 02:00:00+01:00              91.0                0.0
...                                     ...                ...
2023-07-14 13:15:00+02:00             391.0                0.0
2023-07-14 13:30:00+02:00             511.0                0.0
2023-07-14 13:45:00+02:00             592.0                0.0
2023-07-14 14:00:00+02:00             526.0                0.0
2023-07-14 14:15:00+02:00             500.0                0.0

[53714 rows x 2 columns]


C:\Users\emmaa\AppData\Local\Temp\ipykernel_26936\1655986711.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(actual_generation)


Actual Generation for AT - B05:
                           Fossil Hard coal                   
                          Actual Aggregated Actual Consumption
2022-01-01 01:00:00+01:00               0.0                0.0
2022-01-01 01:15:00+01:00               0.0                0.0
2022-01-01 01:30:00+01:00               0.0                0.0
2022-01-01 01:45:00+01:00               0.0                0.0
2022-01-01 02:00:00+01:00               0.0                0.0
...                                     ...                ...
2023-07-14 13:30:00+02:00               0.0                0.0
2023-07-14 13:45:00+02:00               0.0                0.0
2023-07-14 14:00:00+02:00               0.0                0.0
2023-07-14 14:15:00+02:00               0.0                0.0
2023-07-14 14:30:00+02:00               0.0                0.0

[53715 rows x 2 columns]


C:\Users\emmaa\AppData\Local\Temp\ipykernel_26936\1655986711.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(actual_generation)
C:\Users\emmaa\AppData\Local\Temp\ipykernel_26936\1655986711.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(actual_generation)
C:\Users\emmaa\AppData\Local\Temp\ipykernel_26936\1655986711.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_data = all_data.append(actual_generation)


KeyboardInterrupt: 

In [55]:
# Download and save the data for each country - Actual Generation per Production Type
for country_code in sorted_country_codes:
    try:
        actual_generation = client.query_generation(country_code=country_code, start=start, end=end)
        print(f"Actual Generation for {country_code}:")
        print(actual_generation)  # Print the data

        actual_generation.index = actual_generation.index.tz_convert('UTC').strftime('%Y-%m-%d %H:%M')
        actual_generation = actual_generation.replace(['N/A', 'n/e'], None)
        actual_generation.to_csv(f'{country_code}_actual_generation.csv')
    except Exception as e:
        logging.error(f'Data extraction (actual_generation) failed for {country_code}: {str(e)}')

Actual Generation for AT:
                                    Biomass                     \
                          Actual Aggregated Actual Consumption   
2023-07-07 02:00:00+02:00             104.0                0.0   
2023-07-07 02:15:00+02:00             104.0                0.0   
2023-07-07 02:30:00+02:00             104.0                0.0   
2023-07-07 02:45:00+02:00             104.0                0.0   
2023-07-07 03:00:00+02:00             104.0                0.0   
...                                     ...                ...   
2023-07-14 13:45:00+02:00              76.0                0.0   
2023-07-14 14:00:00+02:00              76.0                0.0   
2023-07-14 14:15:00+02:00              80.0                0.0   
2023-07-14 14:30:00+02:00              80.0                0.0   
2023-07-14 14:45:00+02:00               NaN                NaN   

                                 Fossil Gas                     \
                          Actual Aggregated Actua

KeyboardInterrupt: 

In [ ]:
logging.info('Script ended at %s', pd.Timestamp.now(tz='UTC'))